<a href="https://colab.research.google.com/github/FelipeMontLopes/Data_Science_Portfolio/blob/main/Analise_Ativos_Base_nova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Ativos


Fontes de informações: Trading View

https://github.com/StreamAlpha/tvdatafeed

https://tvdb.brianthe.dev/

In [119]:
!pip install --upgrade --no-cache-dir git+https://github.com/StreamAlpha/tvdatafeed.git

!pip install pyproject-toml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/StreamAlpha/tvdatafeed.git to /tmp/pip-req-build-3dyjap_5
  Running command git clone --filter=blob:none --quiet https://github.com/StreamAlpha/tvdatafeed.git /tmp/pip-req-build-3dyjap_5
  Resolved https://github.com/StreamAlpha/tvdatafeed.git to commit a7034f04509b67224618917c29272796e4fff858
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
from tvDatafeed import TvDatafeed, Interval
import json 
import pandas as pd
import numpy as np

import scipy.stats
from scipy.stats import norm

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from math import sqrt

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [121]:
#login no TradingView
tv = TvDatafeed()

In [122]:
#Busca de ativos por ativo e Exchange

search = pd.json_normalize(tv.search_symbol('BTC','BITSTAMP'))
search.head()

,symbol,description,type,exchange,currency_code,currency-logoid,base-currency-logoid,provider_id,is_primary,typespecs
0,BTCUSD,Bitcoin / U.S. dollar,spot,BITSTAMP,USD,country/US,crypto/XTVCBTC,bitstamp,True,[crypto]
1,BTCEUR,Bitcoin / Euro,spot,BITSTAMP,EUR,country/EU,crypto/XTVCBTC,bitstamp,False,[crypto]
2,BTCUSDT,Bitcoin / Tether,spot,BITSTAMP,USDT,crypto/XTVCUSDT,crypto/XTVCBTC,bitstamp,False,[crypto]
3,BTCGBP,Bitcoin / British Pound,spot,BITSTAMP,GBP,country/GB,crypto/XTVCBTC,bitstamp,False,[crypto]
4,BTCPAX,Bitcoin / Paxos Standard,spot,BITSTAMP,PAX,crypto/XTVCUSDPPAXDOLLAR,crypto/XTVCBTC,bitstamp,True,[crypto]


In [123]:
# Consulta dos dados
df1 = tv.get_hist(symbol='BTCUSD',exchange='BITSTAMP',interval=Interval.in_daily,n_bars=3000)

In [124]:


# Excluindo coluna symbol and volume
df1 = df1.drop(['symbol','volume'], axis=1)

# Criando as variáveis
df1["Retorno"] = df1["close"].pct_change(1)

# Volatilidade
df1["Vol_d"] = df1["Retorno"].rolling(20).std()
df1["Vol_y"] = df1["Retorno"].rolling(20).std()*np.sqrt(252)

# Proporção do corpo do candle em relacão ao range do dia
df1["Prop"] = (df1["close"]-df1["open"])/(df1["high"]-df1["low"])

# O Relative Strength Levy

df1["MA5"] = df1["close"].rolling(5).mean()
df1["MA10"] = df1["close"].rolling(10).mean()
df1["MA20"] = df1["close"].rolling(20).mean()
df1["MA52"] = df1["close"].rolling(52).mean()

df1["RSL5"] =(df1["close"]/df1["MA5"]-1)
df1["RSL10"] =(df1["close"]/df1["MA10"]-1)
df1["RSL20"] =(df1["close"]/df1["MA20"]-1)
df1["RSL52"] =(df1["close"]/df1["MA52"]-1)

df1["Std5"] = df1["Retorno"].rolling(5).std()
df1["Std10"] = df1["Retorno"].rolling(10).std()
df1["Std20"] = df1["Retorno"].rolling(20).std()
df1["Std52"] = df1["Retorno"].rolling(52).std()

# Skewness e Kurtosis
periodo = 20
df1["Skewness"] = df1["Retorno"].rolling(periodo).skew()
df1["Kurtosis"] = df1["Retorno"].rolling(periodo).kurt()
df1["Ret_mi"] = df1["Retorno"].rolling(periodo).mean()


#Apagando NaN
df1.dropna(inplace = True)

# Data Frame Final

df1

,open,high,low,close,Retorno,Vol_d,Vol_y,Prop,MA5,MA10,...,RSL10,RSL20,RSL52,Std5,Std10,Std20,Std52,Skewness,Kurtosis,Ret_mi
datetime,,,,,,,,,,,,,,,,,,,,,
2015-02-10,220.91,222.00,215.00,220.42,-0.002173,0.045177,0.717169,-0.070000,223.374,225.407,...,-0.022124,-0.058595,-0.158638,0.021784,0.034324,0.045177,0.066314,-0.308082,0.098756,-0.000525
2015-02-11,220.42,223.43,217.99,219.55,-0.003947,0.044587,0.707789,-0.159926,222.610,224.669,...,-0.022785,-0.059393,-0.156191,0.015839,0.030629,0.044587,0.066227,-0.211457,0.226028,-0.002245
2015-02-12,219.57,223.09,218.01,223.00,0.015714,0.044767,0.710659,0.675197,221.550,223.122,...,-0.000547,-0.042429,-0.136850,0.013348,0.025339,0.044767,0.066212,-0.270639,0.190545,-0.001365
2015-02-13,223.02,242.35,221.37,237.00,0.062780,0.044473,0.705985,0.666349,224.174,224.023,...,0.057927,0.020358,-0.076143,0.030365,0.029840,0.044473,0.066557,-0.294744,0.203624,-0.001554
2015-02-14,237.00,259.31,236.90,256.99,0.084346,0.048176,0.764765,0.892012,231.392,227.022,...,0.132005,0.106021,0.006536,0.040033,0.038821,0.048176,0.067520,-0.116785,-0.093057,0.001435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-03,23469.00,23477.00,22000.00,22359.00,-0.047175,0.030810,0.489091,-0.751523,23217.600,23412.700,...,-0.045005,-0.049724,-0.014736,0.024804,0.020340,0.030810,0.028214,1.452134,3.719007,0.001550
2023-03-04,22359.00,22401.00,22170.00,22347.00,-0.000537,0.030790,0.488782,-0.051948,22988.400,23228.600,...,-0.037953,-0.051359,-0.018937,0.025001,0.020483,0.030790,0.028042,1.438496,3.701605,0.001704
2023-03-05,22347.00,22652.00,22212.00,22428.00,0.003625,0.030791,0.488786,0.184091,22846.800,23077.500,...,-0.028144,-0.049208,-0.018344,0.025380,0.020757,0.030791,0.027248,1.419112,3.660335,0.001883


In [125]:
# Plot - Série de Preços e indicadores

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1.index
                             , open = df1["open"], high = df1["high"]
                             , low = df1["low"], close = df1["close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.update_layout(height = 600, width = 1000
                  , title_text = "Série Histórica "  
                  , font_color = "blue"
                  , title_font_color = "black"
                  , yaxis_title = "Price Close"
                  , legend_title = "Indexes"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )

fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = [
                dict(count = 4,
                     label = "4m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 6,
                     label = "6m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 1,
                     label = "1y",
                     step = "year",
                     stepmode = "backward"),
                dict(step = "all")
            ]),
        type = "date")
    , yaxis = dict(autorange = True, fixedrange = False)
    )


fig.update_layout(hovermode = "x")

fig.show()

In [126]:
# Critérios de Trading e análises

#cálculando média histórica da vol
estatisticas = df1["Vol_d"].describe()
mean_Vol = estatisticas.loc["mean"]*100

# Critérios de Skewness e Kurtosis

skewness_bom = 0
kurtosis = 3
Ret_mi = 0



# Plot - Série de Preços e indicadores
fig = make_subplots(rows = 4, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Vol_d"]*100
                         , name = "Vol", line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_hline(y = mean_Vol, line_width = 1, line_dash = "dash", line_color = "red"
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Skewness"]
                         , name = "Skewness", line = dict(color = "blue"))
              , row = 2, col = 1)

fig.add_hline(y = skewness_bom, line_width = 1, line_dash = "dash", line_color = "red"
              , row = 2, col = 1)


fig.add_trace(go.Scatter(x = df1.index, y = df1["Ret_mi"]
                         , name = "Ret_mi", line = dict(color = "blue"))
              , row = 3, col = 1)

fig.add_hline(y = Ret_mi, line_width = 1, line_dash = "dash", line_color = "red"
              , row = 3, col = 1)


fig.add_trace(go.Scatter(x = df1.index, y = df1["Kurtosis"]
                         , name = "Kurtosis", line = dict(color = "blue"))
              , row = 4, col = 1)

fig.add_hline(y = kurtosis, line_width = 1, line_dash = "dash", line_color = "red"
              , row = 4, col = 1)



fig.update_layout(height = 700, width = 1000
                  , title_text = "Indicadores "  
                  , font_color = "blue"
                  , title_font_color = "black"
                  , yaxis_title = "Vol_d"
                  , yaxis2_title = "Skewness"
                  , yaxis3_title = "Média"
                  , yaxis4_title = "Kurtosis"
                  , legend_title = "Indicadores"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )

fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = [
                dict(count = 4,
                     label = "4m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 6,
                     label = "6m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 1,
                     label = "1y",
                     step = "year",
                     stepmode = "backward"),
                dict(step = "all")
            ]),
        type = "date")
    , yaxis = dict(autorange = True, fixedrange = False)
    )


fig.update_layout(hovermode = "x")

fig.show()

In [127]:
estatisticas_ano = df1.groupby(df1.index.year)["Retorno"].describe()
estatisticas_ano

,count,mean,std,min,25%,50%,75%,max
datetime,,,,,,,,
2015,325.0,0.002479,0.029017,-0.123484,-0.009448,0.001301,0.015379,0.116990
2016,366.0,0.002521,0.024889,-0.161914,-0.005762,0.001722,0.009538,0.105001
2017,365.0,0.008519,0.049315,-0.164940,-0.012236,0.007749,0.033805,0.269211
2018,365.0,-0.002652,0.043802,-0.163202,-0.021264,-0.000971,0.016867,0.141600
2019,365.0,0.002494,0.037014,-0.136744,-0.013516,0.000899,0.018071,0.184538
2020,366.0,0.004678,0.039672,-0.389801,-0.010920,0.003871,0.019641,0.161322
2021,365.0,0.002178,0.042546,-0.142202,-0.021285,0.000599,0.026826,0.194503
2022,365.0,-0.002248,0.033467,-0.155526,-0.015918,-0.001947,0.013502,0.144946
2023,66.0,0.004902,0.025111,-0.050477,-0.007384,-0.000237,0.015296,0.095493


In [128]:
# Construindo a Distribuição dos Retornos

import plotly.express as px

dsitrib = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

dsitrib = px.histogram(df1["Retorno"])

dsitrib.update_layout(height = 400, width = 900
                  , title_text = "Distribuição dos Retornos "
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Retornos"
                  , yaxis_title = "Frequência"               
                  , font = dict(size = 15, color = "Black")
                 )

# Suporte e Resistência baseado na Volatilidade

 - Anual
 - Semanal

In [129]:
df_srvol = df1.copy()
df_srvol

,open,high,low,close,Retorno,Vol_d,Vol_y,Prop,MA5,MA10,...,RSL10,RSL20,RSL52,Std5,Std10,Std20,Std52,Skewness,Kurtosis,Ret_mi
datetime,,,,,,,,,,,,,,,,,,,,,
2015-02-10,220.91,222.00,215.00,220.42,-0.002173,0.045177,0.717169,-0.070000,223.374,225.407,...,-0.022124,-0.058595,-0.158638,0.021784,0.034324,0.045177,0.066314,-0.308082,0.098756,-0.000525
2015-02-11,220.42,223.43,217.99,219.55,-0.003947,0.044587,0.707789,-0.159926,222.610,224.669,...,-0.022785,-0.059393,-0.156191,0.015839,0.030629,0.044587,0.066227,-0.211457,0.226028,-0.002245
2015-02-12,219.57,223.09,218.01,223.00,0.015714,0.044767,0.710659,0.675197,221.550,223.122,...,-0.000547,-0.042429,-0.136850,0.013348,0.025339,0.044767,0.066212,-0.270639,0.190545,-0.001365
2015-02-13,223.02,242.35,221.37,237.00,0.062780,0.044473,0.705985,0.666349,224.174,224.023,...,0.057927,0.020358,-0.076143,0.030365,0.029840,0.044473,0.066557,-0.294744,0.203624,-0.001554
2015-02-14,237.00,259.31,236.90,256.99,0.084346,0.048176,0.764765,0.892012,231.392,227.022,...,0.132005,0.106021,0.006536,0.040033,0.038821,0.048176,0.067520,-0.116785,-0.093057,0.001435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-03,23469.00,23477.00,22000.00,22359.00,-0.047175,0.030810,0.489091,-0.751523,23217.600,23412.700,...,-0.045005,-0.049724,-0.014736,0.024804,0.020340,0.030810,0.028214,1.452134,3.719007,0.001550
2023-03-04,22359.00,22401.00,22170.00,22347.00,-0.000537,0.030790,0.488782,-0.051948,22988.400,23228.600,...,-0.037953,-0.051359,-0.018937,0.025001,0.020483,0.030790,0.028042,1.438496,3.701605,0.001704
2023-03-05,22347.00,22652.00,22212.00,22428.00,0.003625,0.030791,0.488786,0.184091,22846.800,23077.500,...,-0.028144,-0.049208,-0.018344,0.025380,0.020757,0.030791,0.027248,1.419112,3.660335,0.001883


In [130]:
# Definição dos Alvo

df_srvol["Alvo"] = df_srvol["Retorno"].shift(-1)

# Suporte e Resistência com base na vol Anual

In [131]:
year = "2022"

Upper_Band_12m1d =  df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1] + df_srvol[year]["close"][-1]
Lower_Band_12m1d =  df_srvol[year]["close"][-1] - df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1]

Upper_Band_12m2d =  2*df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1] + df_srvol[year]["close"][-1]
Lower_Band_12m2d =  df_srvol[year]["close"][-1] - 2*df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1]

Upper_Band_12m3d =  3*df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1] + df_srvol[year]["close"][-1]

Lower_Band_12m4d =  df_srvol[year]["close"][-1] - 4*df_srvol[year]["Vol_y"][-1]*df_srvol[year]["close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df_srvol[str(int(year)+1)].index
                             , open = df_srvol[str(int(year)+1)]["open"], high = df_srvol[str(int(year)+1)]["high"]
                             , low = df_srvol[str(int(year)+1)]["low"], close = df_srvol[str(int(year)+1)]["close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m3d, line_width = 2, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m4d, line_width = 2, line_dash = "dash", line_color = "red")

fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) 
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df_srvol.index[0]
                       , end = df_srvol.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df_srvol.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

# Suporte e Resistência com base na vol Semanal

In [132]:
# Identification of the begining of the week
# and attribuition of the closing volatility from friday to the next week

df_srvol["Date"] = df_srvol.index
df_srvol["WeekDay"] = df_srvol["Date"].dt.dayofweek #Monday is day 0
df_srvol = df_srvol.dropna(axis = 0)

WeeklyVol = df_srvol["Vol_y"]/np.sqrt(52)
RefPrice = df_srvol["close"]
df_srvol["WeeklyVol"] = df_srvol["Vol_y"]/np.sqrt(52)
df_srvol["RefPrice"] = df_srvol["close"]

for i in range(1,len(df_srvol)):
    if df_srvol["WeekDay"][i] == 0:
        df_srvol["WeeklyVol"][i] = WeeklyVol[i-1]
        df_srvol["RefPrice"][i] = RefPrice[i-1]
    else:
        df_srvol["WeeklyVol"][i] = df_srvol["WeeklyVol"][i-1]
        df_srvol["RefPrice"][i] = df_srvol["RefPrice"][i-1]

In [133]:

df_srvol["Supply_Band_1d"] = np.round(df_srvol["WeeklyVol"]*df_srvol["RefPrice"] + df_srvol["RefPrice"], 2)
df_srvol["Demand_Band_1d"] = np.round(df_srvol["RefPrice"] - df_srvol["WeeklyVol"]*df_srvol["RefPrice"], 2)
df_srvol["Supply_Band_2d"] = np.round(2*df_srvol["WeeklyVol"]*df_srvol["RefPrice"] + df_srvol["RefPrice"], 2)
df_srvol["Demand_Band_2d"] = np.round(df_srvol["RefPrice"] - 2*df_srvol["WeeklyVol"]*df_srvol["RefPrice"], 2)
df_srvol["Supply_Band_3d"] = np.round(3*df_srvol["WeeklyVol"]*df_srvol["RefPrice"] + df_srvol["RefPrice"], 2)
df_srvol["Demand_Band_3d"] = np.round(df_srvol["RefPrice"] - 3*df_srvol["WeeklyVol"]*df_srvol["RefPrice"], 2)
df_srvol["Supply_Band_4d"] = np.round(4*df_srvol["WeeklyVol"]*df_srvol["RefPrice"] + df_srvol["RefPrice"], 2)
df_srvol["Demand_Band_4d"] = np.round(df_srvol["RefPrice"] - 4*df_srvol["WeeklyVol"]*df_srvol["RefPrice"], 2)




# Weekly S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df_srvol.index
                             , open = df_srvol["open"], high = df_srvol["high"]
                             , low = df_srvol["low"], close = df_srvol["close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )
fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Supply_Band_1d"]
                         , name = "Supply_Band_1d", line = dict(color = "Green", width = 1))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Demand_Band_1d"]
                         , name = "Demand_Band_1d", line = dict(color = "Red", width = 1))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Supply_Band_2d"]
                         , name = "Supply_Band_2d", line = dict(color = "Green", width = 1, dash = "dash"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Demand_Band_2d"]
                         , name = "Demand_Band_2d", line = dict(color = "Red", width = 1, dash = "dash"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Supply_Band_3d"]
                         , name = "Supply_Band_3d", line = dict(color = "Green", width = 1, dash = "dash"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Demand_Band_3d"]
                         , name = "Demand_Band_3d", line = dict(color = "Red", width = 1, dash = "dash"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Supply_Band_4d"]
                         , name = "Supply_Band_4d", line = dict(color = "Green", width = 1, dash = "dash"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_srvol.index, y = df_srvol["Demand_Band_4d"]
                         , name = "Demand_Band_4d", line = dict(color = "Red", width = 1, dash = "dash"))
              , row = 1, col = 1)



fig.update_layout(height = 600, width = 1000
                  , title_text = "Weekly S&D Volatility Zones " 
                  , font_color = "blue"
                  , title_font_color = "black"
                  , yaxis_title = "Adj Close"
                  , legend_title = "Indexes"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df_srvol.index[0]
                       , end = df_srvol.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df_srvol.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = [
                dict(count = 4,
                     label = "4m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 6,
                     label = "6m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 1,
                     label = "1y",
                     step = "year",
                     stepmode = "backward"),
                dict(step = "all")
            ]),
        type = "date")
    , yaxis = dict(autorange = True, fixedrange = False)
    )

fig.show()

# Previsão de Volatilidade


In [183]:
df_prev = df1.copy()
df_prev

previsao = df_prev.iloc[-1].copy()
ultima = df_prev["Vol_y"][-1]

In [184]:
# Construção do alvo
alvo = 5

# Alvo 1 - Volatilidade anualizada em x dias para frente
df_prev["Alvo1"] = df_prev["Vol_y"].shift(-alvo)



df_prev.dropna(inplace = True)
df_prev

,open,high,low,close,Retorno,Vol_d,Vol_y,Prop,MA5,MA10,...,RSL20,RSL52,Std5,Std10,Std20,Std52,Skewness,Kurtosis,Ret_mi,Alvo1
datetime,,,,,,,,,,,,,,,,,,,,,
2015-02-10,220.91,222.00,215.00,220.42,-0.002173,0.045177,0.717169,-0.070000,223.374,225.407,...,-0.058595,-0.158638,0.021784,0.034324,0.045177,0.066314,-0.308082,0.098756,-0.000525,0.767078
2015-02-11,220.42,223.43,217.99,219.55,-0.003947,0.044587,0.707789,-0.159926,222.610,224.669,...,-0.059393,-0.156191,0.015839,0.030629,0.044587,0.066227,-0.211457,0.226028,-0.002245,0.757209
2015-02-12,219.57,223.09,218.01,223.00,0.015714,0.044767,0.710659,0.675197,221.550,223.122,...,-0.042429,-0.136850,0.013348,0.025339,0.044767,0.066212,-0.270639,0.190545,-0.001365,0.673309
2015-02-13,223.02,242.35,221.37,237.00,0.062780,0.044473,0.705985,0.666349,224.174,224.023,...,0.020358,-0.076143,0.030365,0.029840,0.044473,0.066557,-0.294744,0.203624,-0.001554,0.683155
2015-02-14,237.00,259.31,236.90,256.99,0.084346,0.048176,0.764765,0.892012,231.392,227.022,...,0.106021,0.006536,0.040033,0.038821,0.048176,0.067520,-0.116785,-0.093057,0.001435,0.674867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-26,23166.00,23678.00,23068.00,23562.00,0.017314,0.031072,0.493258,0.649180,23607.800,24084.800,...,0.011216,0.065775,0.017614,0.022329,0.031072,0.026929,1.304743,3.472427,0.002174,0.489091
2023-02-27,23560.00,23890.00,23126.00,23493.00,-0.002928,0.030752,0.488169,-0.087696,23468.800,23976.100,...,0.007734,0.056641,0.017575,0.015905,0.030752,0.026963,1.463605,4.032295,0.000953,0.488782
2023-02-28,23497.00,23594.00,23048.00,23136.00,-0.015196,0.030819,0.489242,-0.661172,23308.200,23825.700,...,-0.007952,0.035041,0.018027,0.016039,0.030819,0.027117,1.464964,4.000255,0.000819,0.488786


In [185]:
# Separando os dados com as variavem em x e o alvo em y

# Separando os dados entre treinamento e teste

# Vamos treinar o modelo de 2015 a 2019
start_train = "2015-02-10"
end_train = "2018-12-31"

# Vamos testar o modelo de 2020 a Hoje
start_test = "2019-01-01"
end_test = datetime.today().strftime("%Y-%m-%d")

df_prev_train = df_prev.loc[start_train : end_train]
df_prev_test = df_prev.loc[start_test : end_test]

# Separando os dados com as variaveis em x e o alvo em y
 
manter = ["Retorno","Prop", "RSL5", "RSL10", "RSL20", "RSL52", "Std5", "Std10", "Std20", "Std52" ]

x_train = df_prev_train[manter]
y_train = df_prev_train["Alvo1"]

x_test = df_prev_test[manter]
y_test = df_prev_test["Alvo1"]

In [186]:
print(x_train.shape)
print(y_train.shape)


(1421, 10)
(1421,)


# Treinando o modelo de Redes Neurais

In [187]:
# Treinando o modelo

model = MLPRegressor(hidden_layer_sizes = (20,10,20,20), max_iter = 1000,
                     solver = "adam", verbose = 10, tol = 1e-6, random_state = 42,
                     learning_rate_init = .001, activation = "relu")

model.fit(x_train, y_train) # essa é a linha que treina o modelo

Iteration 1, loss = 0.04404590
Iteration 2, loss = 0.04118187
Iteration 3, loss = 0.03922512
Iteration 4, loss = 0.03850084
Iteration 5, loss = 0.03719391
Iteration 6, loss = 0.03657075
Iteration 7, loss = 0.03554576
Iteration 8, loss = 0.03426038
Iteration 9, loss = 0.03312854
Iteration 10, loss = 0.03182932
Iteration 11, loss = 0.03027563
Iteration 12, loss = 0.02874463
Iteration 13, loss = 0.02718654
Iteration 14, loss = 0.02562584
Iteration 15, loss = 0.02395689
Iteration 16, loss = 0.02244660
Iteration 17, loss = 0.02089905
Iteration 18, loss = 0.01952190
Iteration 19, loss = 0.01812727
Iteration 20, loss = 0.01667795
Iteration 21, loss = 0.01522862
Iteration 22, loss = 0.01404301
Iteration 23, loss = 0.01224431
Iteration 24, loss = 0.01042123
Iteration 25, loss = 0.00898770
Iteration 26, loss = 0.00800172
Iteration 27, loss = 0.00735354
Iteration 28, loss = 0.00691156
Iteration 29, loss = 0.00656239
Iteration 30, loss = 0.00631206
Iteration 31, loss = 0.00615784
Iteration 32, los

MLPRegressor(hidden_layer_sizes=(20, 10, 20, 20), max_iter=1000,
             random_state=42, tol=1e-06, verbose=10)

In [188]:
# Predições para o treinamento e teste

y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

In [189]:
# Avaliaçao dos resultados com Redes Neurais

MAE_train = metrics.mean_absolute_error(y_train, y_pred_train)
R2_train = metrics.r2_score (y_train, y_pred_train) # Coeficiente de determinação
RMSE_train = sqrt(metrics.mean_absolute_error(y_train, y_pred_train))


print("----- Avaliação do treinamento -----")
print("MAE: ", round(MAE_train, 3))
print('R2: ', round(R2_train, 3))
print('RMSE: ', round(RMSE_train, 3))


MAE_test = metrics.mean_absolute_error(y_test, y_pred_test)
R2_test = metrics.r2_score (y_test, y_pred_test) # Coeficiente de determinação
RMSE_test = sqrt(metrics.mean_absolute_error(y_test, y_pred_test))


print("----- Avaliação do treinamento -----")
print("MAE: ", round(MAE_test, 3))
print('R2: ', round(R2_test, 3))
print('RMSE: ', round(RMSE_test, 3))

----- Avaliação do treinamento -----
MAE:  0.066
R2:  0.895
RMSE:  0.258
----- Avaliação do treinamento -----
MAE:  0.076
R2:  0.733
RMSE:  0.276


In [190]:
# Resultados: Real x Previsto em um dataframe

resultados = pd.DataFrame ({ "Real": y_test, "Previsto": y_pred_test})
resultados["Residuos"] = resultados["Real"] - resultados["Previsto"]

resultados.tail()

,Real,Previsto,Residuos
datetime,,,
2023-02-26,0.489091,0.372536,0.116555
2023-02-27,0.488782,0.370067,0.118715
2023-02-28,0.488786,0.369403,0.119383
2023-03-01,0.484471,0.337354,0.147117
2023-03-02,0.331347,0.359071,-0.027724


In [194]:
# Avaliando os resultados

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = resultados.index, y = resultados["Real"].rolling(5).mean()
                         , name = "Real", line = dict(color = "red"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = resultados.index, y = resultados["Previsto"].rolling(5).mean()
                         , name = "Previsto", line = dict(color = "black", width = 1, dash = "dash"))
              , row = 1, col = 1)


fig.update_layout(height = 600, width = 1200
                  , title_text = "Volatilidade - Redes Neurais"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Volatility (%)"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = resultados.index[0]
                       , end = resultados.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in resultados.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)
fig.show()

In [192]:
import math
from tabulate import tabulate
print(tabulate([[ (previsao["Vol_y"])*100 
                 ,( model.predict(previsao[manter].values.reshape(1, -1) ) [0])*100 ]]
               ,headers = ["Última volatilidade medida (%) ", "Nível de volatilidade para 7 dias(%)" ]
               ,tablefmt = 'fancy_grid',stralign='center',numalign='center',floatfmt=".2f"))

╒═══════════════════════════════════╤════════════════════════════════════════╕
│  Última volatilidade medida (%)   │  Nível de volatilidade para 7 dias(%)  │
╞═══════════════════════════════════╪════════════════════════════════════════╡
│               33.13               │                 33.35                  │
╘═══════════════════════════════════╧════════════════════════════════════════╛
